In [1]:
from flask import Flask, request, jsonify
import traceback
import base64
import cv2
import skimage
import time
import joblib
import numpy as np
import pickle
from io import BytesIO
from numpy import argmax
from sklearn.preprocessing import StandardScaler

from skimage.feature import local_binary_pattern
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

app = Flask(__name__)
scaler=StandardScaler()
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
lbp_radius = 3
lbp_n_points = 8 * lbp_radius

# Load the SVM model and the scaler
s = time.time()
with open('model.joblib', 'rb') as f:
    svm_model = joblib.load(f)
print("model loading time:",time.time()-s)

classes = {1:"boot", 2:"sandal", 3:"shoe"}
def preprocess_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = preprocess_input(img)
    return img

def extract_features(img):
    resnet_features = resnet_model.predict(np.expand_dims(img, axis=0))[0]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    lbp = local_binary_pattern(gray, lbp_n_points, lbp_radius, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, lbp_n_points + 3), range=(0, lbp_n_points + 2))
    lbp_features = hist / np.sum(hist)
    height, width = img.shape[:2]
    return np.concatenate((resnet_features, lbp_features, [width, height]))

@app.route('/predict', methods=['POST'])
def predict():
    try:
        s = time.time()
        image_data = request.json['image']
        img_data = base64.b64decode(image_data)
        img = cv2.imdecode(np.frombuffer(img_data, dtype=np.uint8), cv2.IMREAD_COLOR)

        img = preprocess_image(img)
        features = extract_features(img)
        features = scaler.fit_transform(features[np.newaxis])  # Scale the features before making predictions

        prediction = svm_model.predict(features)
#         prediction=argmax(prediction)
        return jsonify({
            'prediction': classes.get(int(prediction),int(prediction)),
            "processing_time": str(time.time()-s)
        })
    except:
        return jsonify({
            'prediction': str(traceback.format_exec()),
            "processing_time": 0
        })

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

2023-05-08 12:39:16.788800: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 12:39:18.930750: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mrizwan/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-05-08 12:39:18.930901: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-08 12:39:37.111588: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: 

model loading time: 0.3363955020904541
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.10.37:5000
Press CTRL+C to quit


1/1 [==============================] - 5s 5s/step


/home/mrizwan/.local/lib/python3.10/site-packages/skimage/feature/texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
127.0.0.1 - - [08/May/2023 12:40:26] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2023 12:40:26] "POST /predict HTTP/1.1" 200 -
